Whitepaper: [CausalML: Python Package for Causal Machine Learning](https://arxiv.org/abs/2002.11631)

Package Github [link](https://github.com/uber/causalml)

Package features:
- Models: eight state of the art algorithms for uplift modeling
    - Core uplift modeling
        - Meta Learners
            - R-Learner
            - X-Learner
            - T-Learner
            - S-Learner
            
        - Uplift Trees
            - KL Divergence
            - Chi-Square
            - Euclidean Distance
            - Contextual Treatment Selection
            
    - Supporting functions
        - Data Generation Process
        - Visualization
        - Feature Engineering
        - Propensity Score Model
        - Propenisty Score Matching
- Outcome variables: can handle regression & classification tasks
- No of treatment groups: can handle multiple treatment groups


What problems can causalml solve?
- Targeting optimization
    - We can use CausalML to target promotions to those with the biggest incrementality. For example, at a cross-sell marketing campaign for existing customers, we can deliver promotions to the customers who would be more likely to use a new product specifically given the exposure to the promotions, while saving inboxes for others. 


- Causal impact analysis
    - We can also use CausalML to analyze the causal impact of a particular event from experimental or observational data, incorporating rich features. For example, we can analyze how a customers cross-sell event affects long term spending on the platform. In this case, it is impractical to set up a randomized test because we do not want to exclude any customers from being able to convert to the new product. Utilizing CausalML, we can run various ML-based causal inference algorithms, and estimate the impact of the cross-sell on the platform.


- Personalization
    - CausalML can be used to personalize engagement. There are multiple options for a company to interact with its customers, such as different product choices in up-sell or messaging channels for communications. One can use CausalML to estimate the effect of each combination for each customer, and provide optimal personalized offers to customers.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from xgboost import XGBRegressor
import warnings

from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.dataset import *
from causalml.metrics import *

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

%matplotlib inline

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
